In [2]:
import sympy as sp

t = sp.Symbol('t')
q1 = sp.Function("q1")(t)
q2 = sp.Function("q2")(t)
q1_dot = sp.diff(q1, t)
q2_dot = sp.diff(q2, t)
q1_ddot = sp.diff(q1_dot, t)
q2_ddot = sp.diff(q2_dot, t)

# Generalized coordinates and velocities
q = [q1, q2]
q_dot = sp.Matrix([q1_dot, q2_dot])
q_ddot = sp.Matrix([q1_ddot, q2_ddot])

# Define the link lengths and masses
m, g = sp.symbols('m g')
l1, l2 = sp.symbols('l1 l2')
J1, J2 = sp.symbols('J1 J2')

In [3]:
from sympy.printing.str import StrPrinter

class CustomStrPrinter(StrPrinter):
    def _print_Pow(self, expr):
        base, exp = expr.as_base_exp()
        if exp == 1:
            return self._print(base)
        elif exp == -1:
            return f"1/({self._print(base)})"
        else:
            return f"{self._print(base)}^{self._print(exp)}"

# Use the custom printer
def custom_pretty(expr):
    return CustomStrPrinter().doprint(expr)

In [15]:
# Actuator parameters
xa, ya = sp.symbols('xa ya') #x, y coordinate of actuator

# Rope connection length along joint 2
kx = l1 * sp.cos(q1) + l2 * sp.cos(q2)
ky = l1 * sp.sin(q1) + l2 * sp.sin(q2)

k = sp.Matrix([[kx],
               [ky]])

dkdq1 = sp.diff(k, q1)
dkdq2 = sp.diff(k, q2)

R = sp.Matrix([[xa - kx],
               [ya - ky]])
Rx = R[0]
Ry = R[1]
r = R/R.norm()

l = sp.sqrt((xa - kx)**2 + (ya - ky)**2)
alpha = sp.atan2((ya - ky),(xa - kx))


A1 = sp.diff(l, q1).simplify()
A2 = sp.diff(l, q2).simplify()
sp.pprint(A1, wrap_line = False)
sp.pprint(A2, wrap_line = False)

A = sp.Matrix([[A1],
               [A2]])

A = sp.simplify(A)
#print(custom_pretty(A))
#print("\n")
#sp.pprint(A, wrap_line=False)

           l₁⋅(-l₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))            
──────────────────────────────────────────────────────────────────────────────────
   _______________________________________________________________________________
  ╱                                     2                                       2 
╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  
            l₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))            
──────────────────────────────────────────────────────────────────────────────────
   _______________________________________________________________________________
  ╱                                     2                                       2 
╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  


In [16]:
h1 = sp.atan2(Rx, Ry)
h1 = l
h2 = alpha
theta = sp.Matrix([[h1],
                   [h2]])

Jh = theta.jacobian(q)
Jh = sp.simplify(Jh)
sp.pprint(Jh, wrap_line = False)

⎡           l₁⋅(-l₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))                          l₂⋅(l₁⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₂(t)) - ya⋅cos(q₂(t)))            ⎤
⎢──────────────────────────────────────────────────────────────────────────────────  ──────────────────────────────────────────────────────────────────────────────────⎥
⎢   _______________________________________________________________________________     _______________________________________________________________________________⎥
⎢  ╱                                     2                                       2     ╱                                     2                                       2 ⎥
⎢╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)    ╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⎥
⎢                                                                                                                                                          

In [17]:
Jh_inv = Jh.inv()
Jh_invtrans = Jh_inv.transpose()

In [18]:
A_theta = Jh_invtrans * A
A_theta = sp.simplify(A_theta)
sp.pprint(A_theta, wrap_line = False)

⎡1⎤
⎢ ⎥
⎣0⎦


# Now to find the Mass, Coriolis & Centrifugal and Gravity matrix




In [19]:
# Positions of the center of mass for each link
# Defined in absolute angles from the horizontal 
x1 = l1 * sp.cos(q1)
y1 = l1 * sp.sin(q1)
x2 = l1 * sp.cos(q1) + l2 * sp.cos(q2)
y2 = l1 * sp.sin(q1) + l2 * sp.sin(q2)

# Velocities of the center of mass for each link
vx1 = sp.diff(x1, t)
vy1 = sp.diff(y1, t)
vx2 = sp.diff(x2, t)
vy2 = sp.diff(y2, t)

In [20]:
# Total kinetic energy
T = 0.5 * m * (vx2**2 + vy2**2)

In [21]:
# Total potential energy
V = m * g * y2

# Lagrangian
L = T - V

# Generalized coordinates and velocities
q = [q1, q2]
q_dot = sp.Matrix([q1_dot, q2_dot])
q_ddot = sp.Matrix([q1_ddot, q2_ddot])
replacements = ()
for i in range(2):
    replacements += ((q[i].diff(t).diff(t), sp.Symbol(f'ddq{i + 1}')),
                    (q[i].diff(t), sp.Symbol(f'dq{i + 1}')),
                    (q[i], sp.Symbol(f'q{i + 1}')))


In [22]:
# Lagrangian equations
eoms = []
for i in range(2):
    L_qi = L.diff(q[i].diff(t)).diff(t) - L.diff(q[i])
    L_qi = L_qi.simplify().subs(replacements)
    eoms.append(L_qi)


# L_q2 = sp.diff(sp.diff(L, q2_dot), t) - sp.diff(L, q2)
# L_q2 = L_q2.simplify().subs({sp.diff(q2_dot, t): q2_ddot})
# L_q3 = sp.diff(sp.diff(L, q3_dot), t) - sp.diff(L, q3)
# L_q3 = L_q3.simplify().subs({sp.diff(q3_dot, t): q3_ddot})
# Pretty print the mass matrix, Coriolis matrix, and equations of motion


def format_lagrange(eom, name):
    eom = str(eom).replace('1.0*','')
    terms = str(eom).split('+')
    formatted = f"{name} = " 
    for i, term in enumerate(terms):
        if i == 0:
            formatted += f"{term.strip()}\n"
        else:
            formatted += f"       + {term.strip()}\n"
    return formatted

L_q1_formatted = format_lagrange(eoms[0], "L_q1")
L_q2_formatted = format_lagrange(eoms[1], "L_q2")

if __name__ == "__main__":
    # Print the formatted equations
    print(L_q1_formatted)
    print(L_q2_formatted)


L_q1 = l1*m*(ddq1*l1
       + ddq2*l2*cos(q1 - q2)
       + dq2**2*l2*sin(q1 - q2)
       + g*cos(q1))

L_q2 = l2*m*(ddq1*l1*cos(q1 - q2)
       + ddq2*l2 - dq1**2*l1*sin(q1 - q2)
       + g*cos(q2))



In [23]:
# Custom printer to replace ** with ^ and remove superscript formatting

from sympy.printing.str import StrPrinter

class CustomStrPrinter(StrPrinter):
    def _print_Pow(self, expr):
        base, exp = expr.as_base_exp()
        if exp == 1:
            return self._print(base)
        elif exp == -1:
            return f"1/({self._print(base)})"
        else:
            return f"{self._print(base)}^{self._print(exp)}"

# Use the custom printer
def custom_pretty(expr):
    return CustomStrPrinter().doprint(expr)

# Generalized velocity and acceleration
q_dot = [q[0].diff(t), q[1].diff(t)]
q_ddot = [q_dot[0].diff(t), q_dot[1].diff(t)]
# Mass matrix (inertia matrix)
M = sp.zeros(2)
M4C = sp.zeros(2)
for i in range(2):
    for j in range(2):
        M4C[i, j] = (T.diff(q_dot[i]).diff(q_dot[j])).simplify()
        M[i, j] = M4C[i, j].subs(replacements)
        # M[i, j] = sp.diff(T, q_dot[i], q_dot[j])
M = sp.nsimplify(M)

# Coriolis matrix
C = sp.zeros(2, 2)
for i in range(2):
    for j in range(2):
        for k in range(2):
            C[i, j] += 0.5 * (M4C[i, j].diff(q[k]) + M4C[i, k].diff(q[j]) - M4C[j, k].diff(q[i])) * q_dot[k]
        C[i, j] = C[i, j].simplify().subs(replacements)
C = sp.nsimplify(C)

# Gravitational force vector
G = sp.zeros(2, 1)
for i in range(2):
    G[i] = V.diff(q[i]).simplify().subs(replacements)
G = sp.nsimplify(G)

if __name__ == "__main__":
    # Print the matrices using the custom printer
    print("\nMass Matrix (M):")
    print(custom_pretty(M))

    print("\nCoriolis Matrix (C):")
    print(custom_pretty(C))

    print("\nGravitational Vector (G):")
    print(custom_pretty(G))


Mass Matrix (M):
Matrix([
[              l1^2*m, l1*l2*m*cos(q1 - q2)],
[l1*l2*m*cos(q1 - q2),               l2^2*m]])

Coriolis Matrix (C):
Matrix([
[                        0, dq2*l1*l2*m*sin(q1 - q2)],
[-dq1*l1*l2*m*sin(q1 - q2),                        0]])

Gravitational Vector (G):
Matrix([
[g*l1*m*cos(q1)],
[g*l2*m*cos(q2)]])


In [35]:
A = (xa - kx)
B = (ya - ky)

h = sp.Matrix([[sp.sqrt(A**2+B**2)],
               [sp.atan2(B, A)]])


h1q1 = sp.simplify(sp.diff(h, q1))
sp.pprint(h1q1, wrap_line = False)

⎡           l₁⋅(-l₂⋅sin(q₁(t) - q₂(t)) + xa⋅sin(q₁(t)) - ya⋅cos(q₁(t)))            ⎤
⎢──────────────────────────────────────────────────────────────────────────────────⎥
⎢   _______________________________________________________________________________⎥
⎢  ╱                                     2                                       2 ⎥
⎢╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⎥
⎢                                                                                  ⎥
⎢         l₁⋅(l₁ + l₂⋅cos(q₁(t) - q₂(t)) - xa⋅cos(q₁(t)) - ya⋅sin(q₁(t)))          ⎥
⎢  ─────────────────────────────────────────────────────────────────────────────   ⎥
⎢                                      2                                       2   ⎥
⎣  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)    ⎦


In [30]:
Jh_inv = sp.Matrix([[(A*l2*sp.cos(q2)+B*l2*sp.sin(q2))/(sp.sqrt(A**2+B**2)*l1*l1*sp.sin(q1-q2)),
                     (A*l2*sp.sin(q2)-B*l2*sp.cos(q2))/(l1*l2*sp.sin(q1-q2))],
                   [(-A*l1*sp.cos(q1)-B*l1*sp.sin(q1))/(sp.sqrt(A**2+B**2)*l1*l1*sp.sin(q1-q2)),
                    (-A*l1*sp.sin(q1)+B*l1*sp.cos(q1))/(l1*l2*sp.sin(q1-q2))]])
Jh_inv = sp.simplify(Jh_inv)
Jh_invtrans = Jh_inv.transpose()
A_theta = Jh_invtrans * A
A_theta = sp.simplify(A_theta)
sp.pprint(A_theta, wrap_line = False)

⎡  l₂⋅(l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)⋅(l₁⋅cos(q₁(t) - q₂(t)) + l₂ - xa⋅cos(q₂(t)) - ya⋅sin(q₂(t)))       -(l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)⋅(l₁ + l₂⋅cos(q₁(t) - q₂(t)) - xa⋅cos(q₁(t)) - ya⋅sin(q₁(t)))    ⎤
⎢─────────────────────────────────────────────────────────────────────────────────────────────────────────  ────────────────────────────────────────────────────────────────────────────────────────────────────────⎥
⎢       _______________________________________________________________________________                           _______________________________________________________________________________                   ⎥
⎢  2   ╱                                     2                                       2                           ╱                                     2                                       2                    ⎥
⎢l₁ ⋅╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)) - ya)  + (l₁⋅cos(q₁(t)) + l₂⋅cos(q₂(t)) - xa)  ⋅sin(q₁(t) - q₂(t))  l₁⋅╲╱  (l₁⋅sin(q₁(t)) + l₂⋅sin(q₂(t)